<a href="https://colab.research.google.com/github/jeshraghian/snntorch/blob/master/snntorch/examples/tutorial_3_feedforward_snn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/snntorch_alpha_w.png?raw=true' width="400">

# snnTorch - A Feedforward Spiking Neural Network
## Tutorial 3
### By Jason K. Eshraghian (www.jasoneshraghian.com)

<a href="https://colab.research.google.com/github/jeshraghian/snntorch/blob/tutorials/examples/tutorial_2_neurons.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

The snnTorch tutorial series is based on the following paper. If you find these resources or code useful in your work, please consider citing the following source:

> <cite> Jason K. Eshraghian, Max Ward, Xinxin Wang, Gregor Lenz, Girish Dwivedi, Mohammed Bennamoun, Doo Seok Jeong, and Wei D. Lu. "Training Spiking Neural Networks Using Lessons From Deep Learning". arXiv, 2021. </cite>


# Introduction
In this tutorial, you will:
* Learn how to simplify the leaky integrate-and-fire LIF) neuron to make it deep learning-friendly
* Implement a feedforward spiking neural network (SNN)

Install the latest PyPi distribution of snnTorch by clicking into the following cell and pressing `Shift+Enter`.

In [ ]:
!pip install snntorch

In [ ]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title Plotting Settings
def plot_cur_mem_spk(cur, mem, spk, thr_line=False, vline=False, title=False, ylim_max1=1.25, ylim_max2=1.25):
  # Generate Plots
  fig, ax = plt.subplots(3, figsize=(8,6), sharex=True, 
                        gridspec_kw = {'height_ratios': [1, 1, 0.4]})

  # Plot input current
  ax[0].plot(cur, c="tab:orange")
  ax[0].set_ylim([0, ylim_max1])
  ax[0].set_xlim([0, 200])
  ax[0].set_ylabel("Input Current ($I_{in}$)")
  if title:
    ax[0].set_title(title)

  # Plot membrane potential
  ax[1].plot(mem)
  ax[1].set_ylim([0, ylim_max2]) 
  ax[1].set_ylabel("Membrane Potential ($U_{mem}$)")
  if thr_line:
    ax[1].axhline(y=thr_line, alpha=0.25, linestyle="dashed", c="black", linewidth=2)
  plt.xlabel("Time step")

  # Plot output spike using spikeplot
  splt.raster(spk, ax[2], s=400, c="black", marker="|")
  if vline:
    ax[2].axvline(x=vline, ymin=0, ymax=6.75, alpha = 0.15, linestyle="dashed", c="black", linewidth=2, zorder=0, clip_on=False)
  plt.ylabel("Output spikes")
  plt.yticks([]) 

  plt.show()

def plot_snn_spikes(spk_in, spk1_rec, spk2_rec, title):
  # Generate Plots
  fig, ax = plt.subplots(3, figsize=(8,7), sharex=True, 
                        gridspec_kw = {'height_ratios': [1, 1, 0.4]})

  # Plot input spikes
  splt.raster(spk_in[:,0], ax[0], s=0.03, c="black")
  ax[0].set_ylabel("Input Spikes")
  ax[0].set_title(title)

  # Plot hidden layer spikes
  splt.raster(spk1_rec.reshape(num_steps, -1), ax[1], s = 0.05, c="black")
  ax[1].set_ylabel("Hidden Layer")

  # Plot output spikes
  splt.raster(spk2_rec.reshape(num_steps, -1), ax[2], c="black", marker="|")
  ax[2].set_ylabel("Output Spikes")
  ax[2].set_ylim([0, 10])

  plt.show()

# 1. Simplifying the Leaky Integrate-and-Fire Neuron Model
In the previous tutorial, we designed our own LIF neuron model. But we found that it was quite complex, and it added an array of hyperparameters to tune, including $R$, $C$, $\Delta t$, $U_{\rm thr}$, and the choice of reset mechanism. This is a lot to keep track of, and will grow more cumbersome when we scale this up to a large-scale SNN. So let's make a few simplfications. 

## 1.1 The Decay Rate: $\beta$

In the previous tutorial, we used the Euler method to derive the following solution to the passive membrane model:

$$U(t+\Delta t) = (1-\frac{\Delta t}{\tau})U(t) + \frac{R}{\tau} I_{\rm in}(t) \tag{1}$$

Now assume there is no input current, $I_{\rm in}(t)=0 A$:

$$U(t+\Delta t) = (1-\frac{\Delta t}{\tau})U(t) \tag{2}$$

Also recall, we analytically solved for a time-dependent form of the above equation assuming no current injection: 

$$U(t) = U_0e^{-\frac{t}{\tau}} \tag{3} $$

Where $U_0$ is the initial membrane potential at $t=0$. Assume the time-dependent equation is computed at discrete steps of $t, (t+\Delta t), (t+2\Delta t)~...$, then we can find the ratio of membrane potential between subsequent steps using:

$$\beta = \frac{U_0e^{-\frac{t+\Delta t}{\tau}}}{U_0e^{-\frac{t}{\tau}}} = \frac{U_0e^{-\frac{t + 2\Delta t}{\tau}}}{U_0e^{-\frac{t+\Delta t}{\tau}}} =~~...$$
$$\implies \beta = e^{-\frac{\Delta t}{\tau}} \tag{4}$$

Here, $\beta$ is referred to as the decay rate of the membrane potential, or the inverse time constant. $\beta$ from $(4)$ can be substituted into $(1-\Delta t/\tau)$ from $(2)$ to give:

$$U(t+\Delta t) = \beta U(t)$$

## 1.2 Weighted Input Current
Next, consider the contribution of the input current to the membrane potential, $(R/\tau)I_{\rm in}(t)$, from $(1)$. Assume the input current is sourced from a pre-synaptic neuron. What this means is that the magnitude of the current is weighted by the synaptic strength between the pair of neurons: $W\times I_{\rm in}$. We will assume the effect of $R/\tau$ is subsumed into $W$:

$$U(t+\Delta t) = \beta U(t) + WI_{\rm in}(t) \tag{5}$$

## 1.3 Discretized Time Steps
As we sequentially compute each value of $U(t)$ step-by-step, we can normalize $\Delta t=1$ and represent time discretization in the following way:

$$U[t+1] = \beta U[t] + WI_{\rm in}[t] \tag{6}$$

Note that this will alter the calculation of $\beta$ in $(4)$. Furthermore, we are now assuming that each time bin $t$ is brief enough such that a neuron may only emit a maximum of one spike.


## 1.4 Spiking and Reset
Finally, we introduce the spiking and reset mechanisms. Recall that if the membrane exceeds the threshold, then the neuron emits an output spike: 

$$S_{\rm out}[t] = \begin{cases} 1, &\text{if}~U[t] > U_{\rm thr} \\
0, &\text{otherwise}\end{cases} \tag{7}$$

If a spike is triggered, we wish to reset the membrane potential. The *reset-by-subtraction* is modeled by:

$$U[t+1] = \underbrace{\beta U[t]}_\text{decay} + \underbrace{WI_{\rm in}[t+1]}_\text{input} - \underbrace{S[t+1]U_{\rm thr}}_\text{reset} \tag{8}$$

As $W$ is a learnable parameter, and $U_{\rm thr}$ is often just set to $1$ (though can be tuned), this leaves the decay rate $\beta$ as the only hyperparameter that needs to be specified. This completes the painful part of this tutorial. Let's code this neuron together:

In [ ]:
def leaky_integrate_and_fire(mem, cur, W, beta, threshold=1):
  spk = (mem > threshold) # if membrane exceeds threshold, spk=1, else, 0
  mem = beta * mem + W*cur - spk*threshold
  return spk, mem

To set $\beta$, we have the option of either using $(4)$ to define it, or hard-coding it directly.

In [ ]:
# set neuronal parameters
time_step = torch.tensor(1e-3)
tau = torch.tensor(5e-3)
beta = torch.exp(-time_step/tau)

print(f"The decay rate is: {beta:.3f}")

Let's run a quick simulation to check the neuron behaves correctly in response to a step current input.

In [ ]:
num_steps = 200

# Small step current input
cur_in = torch.cat((torch.zeros(10), torch.ones(190)*0.5), 0)
mem = torch.zeros(1)
spk_out = torch.zeros(1)
mem_rec = []
spk_rec = []

# neuron parameters
W = 0.4
beta = 0.819

# neuron simulation
for step in range(num_steps):
  spk, mem = leaky_integrate_and_fire(mem, cur_in[step], W=W, beta=beta)
  mem_rec.append(mem)
  spk_rec.append(spk)

# convert lists to tensors
mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

plot_cur_mem_spk(cur_in, mem_rec, spk_rec, thr_line=1,
                 title="LIF Neuron Model With Weighted Step Current")

# 2. `Leaky` Neuron Model in snnTorch

In a very similar way to `snn.Lapicque`, our simplified LIF neuron can be instantiated using `snn.Leaky`:



In [ ]:
lif1 = snn.Leaky(beta=0.8)

To use this neuron: 

**Inputs**
* `cur_in`: each element of $W\times I_{\rm in}$ is sequentially passed as an input
* `mem`: the membrane potential, previously $U[t]$, is also passed as input.

**Outputs**
* `spk_out`: output spike $S_{\rm out}[t+1]$ at the next time step ('1' if there is a spike; '0' if there is no spike)
* `mem`: membrane potential $U[t+1]$ at the next time step

These all need to be of type `torch.Tensor`. Note that here, we assume the input current has already been weighted before passing into the `snn.Leaky` neuron. This will be made clear why when we construct a network-scale model.


In [ ]:
# Small step current input
cur_in = torch.cat((torch.zeros(10), torch.ones(190)*0.21), 0)
mem = torch.zeros(1)
spk = torch.zeros(1)
mem_rec = []
spk_rec = []

# neuron simulation
for step in range(num_steps):
  spk, mem = lif1(cur_in[step], mem)
  mem_rec.append(mem)
  spk_rec.append(spk)

# convert lists to tensors
mem_rec = torch.stack(mem_rec)
spk_rec = torch.stack(spk_rec)

plot_cur_mem_spk(cur_in, mem_rec, spk_rec, thr_line=1, ylim_max1=0.5,
                 title="snn.Leaky Neuron Model")

This model has the same optional input arguments of `reset_mechanism` and `threshold` as described for Lapicque's neuron model.

# 2. A Feedforward Spiking Neural Network

So far, we have only considered how one neuron reacts to a single input stimulus. snnTorch makes it extremely straightforward to scale this up to a deep neural network. Here, we will create a 3-layer fully-connected neural network of dimensions 784-1000-10.

Compared to our simulations so far, each neuron will now integrate over many more incoming input spikes. 


<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial2/2_8_fcn.png?raw=true' width="600">
</center>

We will use PyTorch to form the connections between neurons, and snnTorch to create the neurons. First, we must initialize all of our layers.


In [ ]:
# layer parameters
num_inputs = 784
num_hidden = 1000
num_outputs = 10
beta1 = 0.99
beta2 = 0.99

# initialize layers
fc1 = nn.Linear(num_inputs, num_hidden)
lif1 = snn.Leaky(beta=beta1)
fc2 = nn.Linear(num_hidden, num_outputs)
lif2 = snn.Leaky(beta=beta2)

Next, initialize the hidden variables and outputs of each spiking neuron. 
As your networks increase in size, this will become a tedious process. So we can call a static method `init_leaky()` to take care of this. All neurons in snnTorch have their own initialization methods that follow this same syntax, e.g., `init_lapicque()`. The shape of the hidden states are automatically initialized based on the input data dimensions during the first forward pass. 

In [ ]:
# Initialize hidden states
mem1 = lif1.init_leaky()
mem2 = lif2.init_leaky()

# record outputs
mem2_rec = []
spk1_rec = []
spk2_rec = []

Create an input spike train to pass to the network. There are 200 time steps to simulate across 784 input neurons, i.e., the input originally has dimensions of $200 \times 784$. 

However, neural nets typically process data in minibatches. In snnTorch, we use time-first dimensionality: 

[$time \times batch\_size \times feature\_dimensions$]

So 'unsqueeze' the input along `dim=1` to indicate we have 'one batch' of data. The dimensions of this input tensor must be 200 $\times$ 1 $\times$ 784:

In [ ]:
spk_in = spikegen.rate_conv(torch.rand((200, 784))).unsqueeze(1)
print(f"Dimensions of spk_in: {spk_in.size()}")

Now it's finally time to run a full simulation. 
An intuitive way to think about how PyTorch and snnTorch work together is that PyTorch routes the neurons together, and snnTorch loads the results into spiking neuron models. In terms of coding up a network, these spiking neurons can be treated like time-varying activation functions.

Here is a sequential account of what's going on:

*  The $i^{th}$ input to the $j^{th}$ neuron from `spk_in` is weighted by the parameters initialized in `nn.Linear`: $I_{i} \times W_{ij}^{(0)}$
* This generates the input term from Equation $(8)$, contributing to $U[t+1]$ of the spiking neuron
* If $U[t+1] > U_{\rm thr}$, then a spike is triggered from this neuron
* This spike is weighted by the second layer weight, and the above process is repeated for all inputs, weights, and neurons.
* If there is no spike, then nothing is passed to the post-synaptic neuron.

The only difference from our simulations thus far is that we are now scaling the input current with a weight generated by `nn.Linear`. 

In [ ]:
# network simulation
for step in range(num_steps):
    cur1 = fc1(spk_in[step])
    spk1, mem1 = lif1(cur1, mem1)
    cur2 = fc2(spk1)
    spk2, mem2 = lif2(cur2, mem2)

    mem2_rec.append(mem2)
    spk1_rec.append(spk1)
    spk2_rec.append(spk2)

# convert lists to tensors
mem2_rec = torch.stack(mem2_rec)
spk1_rec = torch.stack(spk1_rec)
spk2_rec = torch.stack(spk2_rec)

plot_snn_spikes(spk_in, spk1_rec, spk2_rec, "Fully Connected Spiking Neural Network")

> If you run into errors, then try re-initializing your networks and parameters.


At this stage, the spikes don't have any real meaning. The inputs and weights are all randomly initialized, and no training has taken place. But the spikes  should appear to be propagating from the first layer through to the output!

We can also use `spikeplot.spike_count` to generate a spike counter of the output layer. The following animation will take some time to generate. <br>

> Note: if you are running the notebook locally on your desktop, please uncomment the line below and modify the path to your ffmpeg.exe



In [ ]:
from IPython.display import HTML

fig, ax = plt.subplots(facecolor='w', figsize=(12, 7))
labels=['0', '1', '2', '3', '4', '5', '6', '7', '8','9']
spk2_rec = spk2_rec.squeeze(1).detach().cpu()

# plt.rcParams['animation.ffmpeg_path'] = 'C:\\path\\to\\your\\ffmpeg.exe'

#  Plot spike count histogram
anim = splt.spike_count(spk2_rec, fig, ax, labels=labels, animate=True)
HTML(anim.to_html5_video())
# anim.save("spike_bar.gif")

We can also visualize the membrane potential traces with `spikeplot.traces`. We'll plot 9 out of 10 output neurons. Compare it to the animation and raster plot above to see if you can match the traces to the neuron. 

In [ ]:
# plot membrane potential traces
splt.traces(mem2_rec.squeeze(1), spk=spk2_rec.squeeze(1))

fig = plt.gcf() 

fig.set_size_inches(8, 6)

# Conclusion

That covers how we build a spiking neural network, and how to simplify the leaky integrate-and-fire neuron model. In practice, we will almost always prefer to use `snn.Leaky` over `snn.Lapicque` for training networks, as there is a smaller hyperparameter search space. 

There are a couple of advanced tutorials that describe the `snn.Synaptic` and `snn.Alpha` models, but otherwise, you can feel free to skip on ahead to start training your networks.

For reference, the documentation [can be found here](https://snntorch.readthedocs.io/en/latest/snntorch.html).

## Further Reading
* [snnTorch documentation](https://snntorch.readthedocs.io/en/latest/snntorch.html) of the Lapicque, Leaky, Synaptic, and Alpha models
* [*Neuronal Dynamics:
From single neurons to networks and models of cognition*](https://neuronaldynamics.epfl.ch/index.html) by
Wulfram Gerstner, Werner M. Kistler, Richard Naud and Liam Paninski.
* [Theoretical Neuroscience: Computational and Mathematical Modeling of Neural Systems](https://mitpress.mit.edu/books/theoretical-neuroscience) by Laurence F. Abbott and Peter Dayan